In [1]:
import requests
from bs4 import BeautifulSoup as BS
import config
import mysql.connector
from mysql.connector import errorcode
from datetime import date, datetime, timedelta
import time

headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
numbers_root = 'https://www.the-numbers.com'
db_name = 'Movies'


In [2]:
def connect_to_AWS():
    cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        passwd = config.password
    )
    return cnx

def close_connections():
    cursor.close()
    conn.close()

In [ ]:
# conn = connect_to_AWS()
# cursor = conn.cursor()

In [45]:
## Function to create new DB 
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)

## Try to use the database we are creating.
## If we can't use it, then that means the database doesn't exist
    ## Thus we create the database and switch over to it.
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        conn.database = db_name
    else:
        print(err)
        exit(1)
        
# Function to create tables from a dictionary of tables
# dictionary will have the name of the table as the key and the query to create that table as the value
def create_table(dict_of_tables):
    for table_name in dict_of_tables:
        table_query = dict_of_tables[table_name]
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_query)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")
        
def drop_tables():
    cursor.execute("""DROP TABLE movies""")
    conn.commit()

In [4]:
TABLES = {}
TABLES['movies'] = """CREATE TABLE movies (
        movieId INT NOT NULL AUTO_INCREMENT,
        movieTitle varchar(255),
        movieDomGross INT(10),
        movieIntGross INT(10),
        movieBudget INT(10),
        movieRunTime REAL,
        movieAgeRating varchar(10),
        movieReleaseDate DATE,
        movieProdMethod varchar(30),
        movieGenre varchar(20),
        movieType varchar(30),
        PRIMARY KEY (movieId),
        UNIQUE KEY(movieTitle)
    ) ENGINE=InnoDB"""

TABLES['people'] = """

"""

TABLES['movies_people'] = """CREATE TABLE `movie_people` (
  `movieId` INT NOT NULL,
  `personId` INT NOT NULL,
  PRIMARY KEY (`movieId`, `personId`),
  INDEX `movieId_idx` (`movieId` ASC),
  CONSTRAINT `moivieId`
    FOREIGN KEY (`movieId`)
    REFERENCES `Movies`.`movies` (`movieId`)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT `personId`
    FOREIGN KEY (`personId`)
    REFERENCES `Movies`.`people` (`personId`)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
    ENGINE=InnoDB
"""

In [4]:
# drop_tables()

In [5]:
# create_table(TABLES)
# # cursor.execute("""ALTER TABLE `Movies`.`movies` AUTO_INCREMENT = 101;""")
# conn.commit()

Creating table movies: already exists.


In [5]:
def get_soup(url):
    try:
        page = requests.get(url, timeout=5)
    # include code to do status check
        if page.status_code != 200:
            print(page.status_code)
        else:
#             print(page.status_code)
            soup = BS(page.content,'html.parser')
    except requests.ConnectionError as e:
        print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
        print(str(e))
    except requests.Timeout as e:
        print("OOPS!! Timeout Error")
        print(str(e))
    except requests.RequestException as e:
        print("OOPS!! General Error")
        print(str(e))
    except KeyboardInterrupt:
        print("Someone closed the program") 
        
    
    time.sleep(1)
    return soup

In [56]:
def str_to_int(string):
    return int(string.replace(',','').strip('$'))
    
def str_to_date(string):
    return datetime.strptime(string,"%b %d, %Y").date()

def str_to_time(string):
    return (int(string.strip('minutes')))
    
def get_from_pages(page_start,page_end):
    url = numbers_root+"/movie/budgets/all"
    for page in ['/'+str(num)+'01' for num in range(page_start-1,page_end)]:
        movie_rows = get_movie_rows(url+page)
        for movie_row in movie_rows:
#             print(movie_row)
            movie_info = get_movie_info(movie_row[0])
            if movie_info != 404:
                insert_movie(movie_row[1:]+movie_info)

def get_movie_rows(page_url):
    movie_tuples = []
    soup = get_soup(page_url)
    movie_fins = soup.select('tr td.data')
    movie_exts = soup.select('tr td b a')
    movie_release = soup.select('tr td a')
#     print(movie_release)
#     print(movie_urls)
    for i in range(100):
        ind = i*4
        ind2 = i*2
        if str_to_date(movie_release[ind2].text) < datetime.now().date():
            movie_tuples.append((movie_exts[i]['href'],
                                 movie_exts[i].text,
                                 str_to_date(movie_release[ind2].text),
                                 str_to_int(movie_fins[ind+1].text),
                                 str_to_int(movie_fins[ind+2].text),
                                 str_to_int(movie_fins[ind+3].text)))
    return movie_tuples
#     return [(movie_row['href'],) for movie_row in movie_rows]

## Get runtime, MPAA Rating, Production method, genre, and creative type
def get_movie_info(movie_ext):
    retVal = 404
    try:
        runtime = rating = prod_method = genre = c_type = None
        url = numbers_root + movie_ext
        soup = get_soup(url)
        all_html = soup.select('tr td a')
        full_page = soup.select('tr td')
        
        for time in full_page:
            if time.text.endswith('minutes'):
                runtime = str_to_time(time.text)
                break
        if runtime != None:
            for html in all_html:
                if (rating and prod_method and genre and c_type):
                    retVal = (runtime, rating, prod_method, genre, c_type)

                elif html['href'].startswith('/market/mpaa-rating/'):
                    rating = html.text

                elif html['href'].startswith('/market/genre/'):
                    genre = html.text

                elif html['href'].startswith('/market/production-method/'):
                    prod_method = html.text

                elif html['href'].startswith('/market/creative-type/'):
                    c_type = html.text

        return retVal
    except:
        return retVal
    
def insert_movie(movie_data):
    print(movie_data)
    query = ("""INSERT IGNORE INTO movies
                    (movieTitle,movieReleaseDate,movieBudget,movieDomGross,movieIntGross,
                    movieRunTime,movieAgeRating,movieProdMethod,movieGenre,movieType)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
             """)
    cursor.execute(query,movie_data)
    conn.commit()

In [59]:
# get_from_pages(5,10)

('Green Zone', datetime.date(2010, 3, 12), 100000000, 35497337, 97523020, 114, 'R', 'Live Action', 'Drama', 'Historical Fiction')
('Geostorm', datetime.date(2017, 10, 20), 100000000, 33700160, 220796009, 109, 'PG-13', 'Live Action', 'Action', 'Science Fiction')
('Point Break', datetime.date(2015, 12, 25), 100000000, 28782481, 126704591, 113, 'PG-13', 'Live Action', 'Action', 'Contemporary Fiction')
('In the Heart of the Sea', datetime.date(2015, 12, 11), 100000000, 25020758, 89693309, 121, 'PG-13', 'Live Action', 'Adventure', 'Dramatization')
('Transcendence', datetime.date(2014, 4, 18), 100000000, 23022309, 103039258, 119, 'PG-13', 'Live Action', 'Thriller/Suspense', 'Science Fiction')
('Son of the Mask', datetime.date(2005, 2, 18), 100000000, 17018422, 59918422, 94, 'PG', 'Live Action', 'Adventure', 'Kids Fiction')
('Mortal Engines', datetime.date(2018, 12, 14), 100000000, 15951040, 85287417, 128, 'PG-13', 'Live Action', 'Action', 'Science Fiction')
('Jin lÃ\xadng shÃ\xad san chai', 

('Valkyrie', datetime.date(2008, 12, 25), 90000000, 83107829, 203932174, 120, 'PG-13', 'Live Action', 'Thriller/Suspense', 'Dramatization')
('Pixels', datetime.date(2015, 7, 24), 90000000, 78765986, 244041804, 105, 'PG-13', 'Animation/Live Action', 'Adventure', 'Science Fiction')
('Artificial Intelligence: AI', datetime.date(2001, 6, 29), 90000000, 78616689, 235900000, 145, 'PG-13', 'Animation/Live Action', 'Drama', 'Science Fiction')
('The Haunted Mansion', datetime.date(2003, 11, 26), 90000000, 75817994, 155750628, 88, 'PG', 'Live Action', 'Adventure', 'Kids Fiction')
('The Interpreter', datetime.date(2005, 4, 22), 90000000, 72708161, 162753837, 128, 'PG-13', 'Live Action', 'Thriller/Suspense', 'Contemporary Fiction')
('Percy Jackson: Sea of Monsters', datetime.date(2013, 8, 7), 90000000, 68559554, 200859554, 105, 'PG', 'Animation/Live Action', 'Adventure', 'Fantasy')
('Lara Croft: Tomb Raider: The Cradle of Life', datetime.date(2003, 7, 25), 90000000, 65653758, 157092943, 117, 'PG-1

('The Core', datetime.date(2003, 3, 28), 85000000, 31111260, 74120792, 137, 'PG-13', 'Live Action', 'Action', 'Science Fiction')
('The Greatest Showman', datetime.date(2017, 12, 20), 84000000, 174340174, 386665550, 105, 'PG', 'Live Action', 'Musical', 'Dramatization')
('Scooby-Doo', datetime.date(2002, 6, 14), 84000000, 153294164, 276294164, 86, 'PG', 'Live Action', 'Adventure', 'Kids Fiction')
('Nutty Professor II: The Klumps', datetime.date(2000, 7, 28), 84000000, 123307945, 166307945, 106, 'PG-13', 'Live Action', 'Comedy', 'Contemporary Fiction')
('RED 2', datetime.date(2013, 7, 19), 84000000, 53262560, 141507355, 116, 'PG-13', 'Live Action', 'Action', 'Contemporary Fiction')
('Click', datetime.date(2006, 6, 23), 82500000, 137355633, 237685089, 106, 'PG-13', 'Live Action', 'Comedy', 'Fantasy')
("Charlotte's Web", datetime.date(2006, 12, 15), 82500000, 82985708, 143985708, 97, 'G', 'Live Action', 'Drama', 'Fantasy')
('Jumper', datetime.date(2008, 2, 14), 82500000, 80172128, 222640812

('Little Nicky', datetime.date(2000, 11, 10), 80000000, 39442871, 58270391, 91, 'PG-13', 'Live Action', 'Comedy', 'Fantasy')
('Surrogates', datetime.date(2009, 9, 25), 80000000, 38577772, 119668350, 89, 'PG-13', 'Live Action', 'Action', 'Science Fiction')
('Evolution', datetime.date(2001, 6, 8), 80000000, 38311134, 98341932, 101, 'PG-13', 'Live Action', 'Comedy', 'Contemporary Fiction')
('The Brothers Grimm', datetime.date(2005, 8, 26), 80000000, 37899638, 105299638, 120, 'PG-13', 'Live Action', 'Adventure', 'Fantasy')
('Mars Attacks!', datetime.date(1996, 12, 13), 80000000, 37771017, 101371017, 103, 'PG-13', 'Live Action', 'Comedy', 'Science Fiction')
('The Wild', datetime.date(2006, 4, 14), 80000000, 37384046, 99010667, 82, 'G', 'Digital Animation', 'Adventure', 'Contemporary Fiction')
('Walking with Dinosaurs', datetime.date(2013, 12, 20), 80000000, 36076121, 123368842, 87, 'PG', 'Animation/Live Action', 'Adventure', 'Kids Fiction')
('Thirteen Days', datetime.date(2000, 12, 22), 800

('Tears of the Sun', datetime.date(2003, 3, 7), 75000000, 43632458, 85632458, 109, 'R', 'Live Action', 'Action', 'Contemporary Fiction')
('Lady in the Water', datetime.date(2006, 7, 21), 75000000, 42285169, 72785169, 110, 'PG-13', 'Live Action', 'Drama', 'Fantasy')
('Spanglish', datetime.date(2004, 12, 17), 75000000, 42044321, 54344321, 133, 'PG-13', 'Live Action', 'Comedy', 'Contemporary Fiction')
('Bandits', datetime.date(2001, 10, 12), 75000000, 41523271, 71523271, 123, 'PG-13', 'Live Action', 'Comedy', 'Contemporary Fiction')
('First Knight', datetime.date(1995, 7, 7), 75000000, 37361412, 127361412, 134, 'PG-13', 'Live Action', 'Drama', 'Historical Fiction')
('Sucker Punch', datetime.date(2011, 3, 25), 75000000, 36392502, 89758389, 110, 'R', 'Live Action', 'Action', 'Fantasy')
('Inferno', datetime.date(2016, 10, 28), 75000000, 34343574, 219519367, 121, 'PG-13', 'Live Action', 'Thriller/Suspense', 'Contemporary Fiction')
('Hollywood Homicide', datetime.date(2003, 6, 13), 75000000, 3

('Rock of Ages', datetime.date(2012, 6, 15), 70000000, 38518613, 61031932, 123, 'R', 'Live Action', 'Musical', 'Contemporary Fiction')
('Seven Years in Tibet', datetime.date(1997, 10, 8), 70000000, 37945884, 131445884, 131, 'PG-13', 'Live Action', 'Drama', 'Dramatization')
('Sky Captain and the World of Tomorrow', datetime.date(2004, 9, 17), 70000000, 37760080, 49730854, 107, 'PG', 'Animation/Live Action', 'Adventure', 'Science Fiction')
('The Brave One', datetime.date(2007, 9, 14), 70000000, 36793804, 69792704, 122, 'R', 'Live Action', 'Drama', 'Contemporary Fiction')
('I Spy', datetime.date(2002, 11, 1), 70000000, 33561137, 60279822, 92, 'PG-13', 'Live Action', 'Action', 'Contemporary Fiction')
('Bad Company', datetime.date(2002, 6, 7), 70000000, 30157016, 69157016, 116, 'PG-13', 'Live Action', 'Action', 'Contemporary Fiction')
('The Finest Hours', datetime.date(2016, 1, 29), 70000000, 27569558, 49252761, 116, 'PG', 'Live Action', 'Thriller/Suspense', 'Dramatization')
('Escape Plan',

('Focus', datetime.date(2015, 2, 27), 65000000, 53862963, 168065700, 105, 'PG-13', 'Live Action', 'Romantic Comedy', 'Contemporary Fiction')
('Blade: Trinity', datetime.date(2004, 12, 8), 65000000, 52397389, 131353165, 113, 'R', 'Live Action', 'Action', 'Super Hero')
('Red Dawn', datetime.date(2012, 11, 21), 65000000, 44806783, 48164150, 93, 'PG-13', 'Live Action', 'Action', 'Contemporary Fiction')
('Everest', datetime.date(2015, 9, 18), 65000000, 43482270, 221297061, 121, 'PG-13', 'Live Action', 'Thriller/Suspense', 'Dramatization')
('Resident Evil: Retribution', datetime.date(2012, 9, 14), 65000000, 42345531, 240647629, 95, 'R', 'Live Action', 'Action', 'Science Fiction')
('Fight Club', datetime.date(1999, 10, 15), 65000000, 37030102, 100851705, 139, 'R', 'Live Action', 'Drama', 'Contemporary Fiction')
('Death Race', datetime.date(2008, 8, 22), 65000000, 36316032, 72516819, 110, 'R', 'Live Action', 'Action', 'Science Fiction')
('The Long Kiss Goodnight', datetime.date(1996, 10, 11), 

('The Family Man', datetime.date(2000, 12, 22), 60000000, 75764085, 124715863, 125, 'PG-13', 'Live Action', 'Comedy', 'Contemporary Fiction')
('Contagion', datetime.date(2011, 9, 9), 60000000, 75658097, 137551594, 105, 'PG-13', 'Live Action', 'Thriller/Suspense', 'Science Fiction')
('Any Given Sunday', datetime.date(1999, 12, 22), 60000000, 75530832, 100230832, 164, 'R', 'Live Action', 'Drama', 'Contemporary Fiction')
('The Horse Whisperer', datetime.date(1998, 5, 15), 60000000, 75383563, 186883563, 169, 'PG-13', 'Live Action', 'Drama', 'Contemporary Fiction')
('Coraline', datetime.date(2009, 2, 6), 60000000, 75286229, 126037057, 100, 'PG', 'Stop-Motion Animation', 'Adventure', 'Kids Fiction')
('Deep Blue Sea', datetime.date(1999, 7, 28), 60000000, 73648228, 165048228, 105, 'R', 'Live Action', 'Action', 'Contemporary Fiction')
("Michael Jackson's This Is It", datetime.date(2009, 10, 28), 60000000, 72091016, 252091016, 111, 'PG', 'Live Action', 'Documentary', 'Factual')
('Kangaroo Jack'

('Inkheart', datetime.date(2009, 1, 23), 60000000, 17303424, 66655938, 106, 'PG', 'Live Action', 'Adventure', 'Fantasy')
('Supernova', datetime.date(2000, 1, 14), 60000000, 14218868, 14816494, 90, 'PG-13', 'Live Action', 'Action', 'Science Fiction')
('Flyboys', datetime.date(2006, 9, 22), 60000000, 13090630, 14816379, 139, 'PG-13', 'Live Action', 'Drama', 'Dramatization')
('Winterâ\x80\x99s Tale', datetime.date(2014, 2, 14), 60000000, 12600231, 29575033, 118, 'PG-13', 'Live Action', 'Drama', 'Fantasy')
('Holy Man', datetime.date(1998, 10, 9), 60000000, 12069719, 12069719, 113, 'PG', 'Live Action', 'Comedy', 'Contemporary Fiction')
('Meet Dave', datetime.date(2008, 7, 11), 60000000, 11803254, 50648806, 90, 'PG', 'Live Action', 'Adventure', 'Science Fiction')
('The Great Raid', datetime.date(2005, 8, 12), 60000000, 10166502, 10597070, 132, 'R', 'Live Action', 'Action', 'Historical Fiction')
('Rock Dog', datetime.date(2017, 2, 24), 60000000, 9420546, 24148488, 80, 'PG', 'Digital Animation

In [57]:
# drop_tables()
# create_table(TABLES)

Creating table movies: OK


In [62]:
# close_connections()